In [1]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as  nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
import time
from tqdm.autonotebook import tqdm, trange
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'


#######################Тут не все компании!!!######################################
#######################Тут не все компании!!!######################################
companies = ['SBER','GAZP' , 'LKOH', 'GMKN', 'MGNT',
       'TATN', 'NVTK', 'SNGS', 'PLZL', 'SNGSP',
       'PIKK', 'ROSN', 'SBERP', 'CHMF', 'NLMK',
       'IRAO', 'YNDX', 'ALRS', 'RUAL', 'MTSS',
       'MAGN', 'PHOR', 'RTKM', 'VTBR', 'TATNP',
       'TCSG', 'AGRO', 'AFLT', 'AFKS',
       'TRNFP', 'CBOM',
       'FIVE', 'MOEX',
       'HYDR']
companies = ['SBER', 'SBERP', 'VTBR', 'TATNP','TRNFP', 'MOEX']
######################Тут не все компании!!!##############################################
######################Тут не все компании!!!##############################################

/tmp/ipykernel_32629/2117073850.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
path = '~/Рабочий стол/ДИПЛОМ НОУТБУКИ/DL forcasitng diplom/Данные Мосбиржа/Total_MOEX_with_time_df'
total_df = pd.read_csv(path)

In [3]:
total_df.tradetime = total_df.close
total_df.drop(['close', 'pr_close'], 1, inplace = True)
total_df.rename(columns={'tradetime':'close'}, inplace=True)
total_df = total_df.set_index('tradedate')

/tmp/ipykernel_32629/442606538.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  total_df.drop(['close', 'pr_close'], 1, inplace = True)


In [4]:
max_len = 94740
data = np.zeros((max_len, len(companies)))
for i, company in enumerate(companies):
    series = np.array(total_df[total_df.secid == company].close)[:max_len]
    data[:,i] += series

In [5]:
train_data = data[ : round(data.shape[0]*0.55) ]
val_data = data[ round(data.shape[0]*0.60) : ]

In [6]:
train_data = train_data
val_data = val_data

for i in range(data.shape[1]):
    
    if i != 0:
        train_series = train_data[:,i].reshape(-1, 1)
        val_series = val_data[:,i].reshape(-1, 1)
    
        scaler = StandardScaler()
        train_series = scaler.fit_transform(train_series)
        val_series = scaler.transform(val_series)
    
        train_data[:, i] = train_series.flatten()
        val_data[:, i] = val_series.flatten()
    elif i == 0:
        train_series = train_data[:,i].reshape(-1, 1)
        val_series = val_data[:,i].reshape(-1, 1)
    
        scaler_star = StandardScaler()
        train_series = scaler_star.fit_transform(train_series)
        val_series = scaler_star.transform(val_series)
    
        train_data[:, i] = train_series.flatten()
        val_data[:, i] = val_series.flatten()

data_train = train_data
data_val = val_data

In [7]:
def sample_creator(df, lookback, horizon):
    """
    Функция нарезает данные скользящим окном, 
    lookback=исторический промежуток
    horizon=горизонт прогнозирования
    !Внимание! target переменная должна стоять в первом столбце!
    """
    X = []
    Y = np.array([np.zeros(horizon) for i in range(lookback, df.shape[0]-horizon+1 )])
    
    for i in range(lookback, df.shape[0]-horizon+1):
        x = df[i-lookback : i, :]
        X.append(x)
        y = df[i:i+horizon, 0]
        Y[i-lookback] += y
        
        
    X = torch.Tensor(np.array(X))
    Y = torch.Tensor(Y)
    return X, Y

In [17]:
lookback = 80
horizon = 1

x_train, y_train = sample_creator(data_train, lookback, horizon)
x_val, y_val = sample_creator(data_val, lookback, horizon)

print(f'x_train: {x_train.shape}, y_train: {y_train.shape}\nx_val: {x_val.shape}, y_val: {y_val.shape}')

x_train: torch.Size([52027, 80, 6]), y_train: torch.Size([52027, 1])
x_val: torch.Size([37816, 80, 6]), y_val: torch.Size([37816, 1])


In [18]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset  = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=60, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=60, shuffle=False)

In [19]:
from PatchTST_models import PatchTST, winrate

In [20]:
print(PatchTST.__doc__)


    Входыне данные:
    1) Кол-во рядов т.е. фичей: (M)
    2) Длина ряда: (L)
    3) Длина патча: (P)
    4) Страйд: (S)
    5) Кол-во головок памяти: (num_heads)
    6) Внутреннаяя размерность FFN: (dim_feedforward)
    7) Кол-во слоёв в энкодер блоке: (num_layers)
    8) Размерность энкодера: (d_model)
    9) Dropout: (dropout)
    
    
    


In [21]:
model = PatchTST(M=len(companies),L=80,P=2, S=2, num_heads=8, dim_feedforward=2048, 
                 num_layers=5, d_model=512, dropout=0.1).to(device)

In [ ]:
##### Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=3, verbose=True)

epochs = 50
early_stop_count = 0
min_val_loss = float('inf')

train_loss_per_epoch = []
val_loss_per_epoch = []
train_winrate = []
val_winrate = []



pbar = trange(epochs, desc="Epoch:")
since = time.time()

for epoch in pbar:
    model.train()
    train_losses = 0
    train_win = 0
    for batch in tqdm(train_loader, leave=False, desc="Train iter:"):
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        outputs = outputs[:,[0]]
        loss = criterion(outputs, y_batch)
        loss.backward()
        train_losses += loss.item()
        optimizer.step()
        train_loss = train_losses/len(train_loader)
        train_win += winrate(y_batch, outputs)
    train_loss_per_epoch.append(train_loss)    
    train_winrate.append(train_win / len(train_loader))
    
    # Validation
    model.eval()
    val_losses = 0
    val_win = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, leave=False, desc="Val iter:"):
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            outputs = outputs[:,[0]]
            loss = criterion(outputs, y_batch)
            val_losses += loss.item()
            val_win += winrate(y_batch, outputs)
    val_loss = val_losses/len(val_loader)
    val_loss_per_epoch.append(val_loss)
    val_winrate.append(val_win / len(val_loader))
    
    scheduler.step(val_loss)
    if val_loss < 0.00012:
        torch.save(model.state_dict(), f'ВЕСА/PatchTST(M F=TI) 1D|W={val_win / len(val_loader)}, VL={val_loss}' )
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 15:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}","/",f"{epochs},Train Loss: {train_loss:.6f} Validation Loss: {val_loss:.6f}")
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    

Epoch::   0%|          | 0/50 [00:00<?, ?it/s]

Train iter::   0%|          | 0/868 [00:00<?, ?it/s]

Val iter::   0%|          | 0/631 [00:00<?, ?it/s]

Epoch 1 / 50,Train Loss: 0.195549 Validation Loss: 2.036355


Train iter::   0%|          | 0/868 [00:00<?, ?it/s]

...

Epoch 12 / 50,Train Loss: 0.026420 Validation Loss: 0.608245


Train iter::   0%|          | 0/868 [00:00<?, ?it/s]

Val iter::   0%|          | 0/631 [00:00<?, ?it/s]

Epoch 13 / 50,Train Loss: 0.025062 Validation Loss: 0.666174


Train iter::   0%|          | 0/868 [00:00<?, ?it/s]